In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import fake_useragent
import re
import unicodedata
import warnings
warnings.filterwarnings("ignore")

ua = fake_useragent.UserAgent()

In [2]:
def get_url(category, tot_pages):
    for page in range(0,tot_pages+1):
        if page % log_frequency == 0:
            print(f'Log: working over page {page}...')       
        
        url_main = f'https://petrasport.ru/{category}/p/{page}/'
        response = requests.get(url_main, headers = {'user-agent': ua.random}, verify=False)
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all(class_ = 'product-name')
        for item in data:
            url = item.find('a', href = True)
            yield ('https://petrasport.ru' + url['href'])

In [3]:
def get_item(category, tot_pages):

    for url_item in get_url(category, tot_pages):
        response = requests.get(url_item, headers = {'user-agent': ua.random}, verify=False)
        soup = BeautifulSoup(response.content, 'lxml')

        try:
            title = soup.find('h1').text
            title = title.replace('\r', '').replace('\t', '').replace('\n', '').strip()
        except:
            title = ''

        article = ''

        try:
            image = soup.find(class_ = 'product-image')
            url = 'https://petrasport.ru' + image.find('a')['href']

        except:
            print(f'Log: no image for {article} found...')
            url = ''

        try:
            char = soup.find_all(class_ = 'even')
            char_string = ''
            for ch in char:
                children = ch.children
                for child in children:
                    if bool(child.find(class_ = 'popup-param')):
                        pass
                    else:
                        char_string += child.text
                        char_string += ' '
            
            char = soup.find_all(class_ = 'add')
            for ch in char:
                children = ch.children
                for child in children:
                    if bool(child.find(class_ = 'popup-param')):
                        pass
                    else:
                        char_string += child.text
                        char_string += ' '
            chars = unicodedata.normalize('NFKD',char_string).strip()
        except:
            chars = ''


        desc = ''

        try:
            price = soup.find(class_ = 'price-current').text
            price = unicodedata.normalize('NFKD',price).replace(' ', '').replace('руб.', '')
        except:
            price =''

        try:
            cat = soup.find(class_ = 'site-path').text
        except:
            cat = []
        try:
            cat_1 = cat.split('/')[0].strip()
        except:
            cat_1 = ''
        try:
            cat_2 = cat.split('/')[1].strip()
        except:
            cat_2 = ''
        try:
            cat_3 = cat.split('/')[2].strip()
        except:
            cat_3 = ''

        yield  title, article, price, cat_1, cat_2, cat_3, url, desc, chars

In [4]:
df_columns = ['title', 'article','price', 'cat_1', 'cat_2', 'cat_3','url', 
              'description', 'caracteristics']

df = pd.DataFrame(columns = df_columns)
save_frequency = 300
log_frequency = 20

In [5]:
def to_csv(category, tot_pages):
    counter = 0  

    for item in get_item(category, tot_pages):
            data = []
            for i in range(len(df_columns)):
                data.append(item[i])
            df.loc[len(df.index)] = data
            #time.sleep(0.05)

            counter += 1
            if counter % save_frequency == 0:
                df.to_csv('petrasport.csv')
                print(f'{counter} items saved in csv..')
    df.to_csv('petrasport.csv')

    print(f'\nTotal items saved: {counter}')  

In [6]:
to_csv('begovye-dorozhki', 86)

Log: working over page 0...
300 items saved in csv..
Log: working over page 20...
Log: no image for  found...
600 items saved in csv..
Log: working over page 40...
900 items saved in csv..
1200 items saved in csv..
Log: working over page 60...
1500 items saved in csv..
Log: working over page 80...

Total items saved: 1736


In [7]:
def show_image(val):
    return '<a href="{}"><img src="{}" width=10000></img></a>'.format(val, val)
df.iloc[44:46].style.format({'url': show_image, **{'width': '100px'}})

,title,article,price,cat_1,cat_2,cat_3,url,description,caracteristics
44,Беговая дорожка Raizer TM1370D,,18420,Главная,Беговые дорожки,Raizer,,,"Вес 36 кг Страна изготовления Китай Складной Да электрическая Максимальный вес пользователя 100 кг Ширина бегового полотна 37 см до 10 км/ч Консоль LCD-дисплей Количество программ тренировки 9 есть Возможность профессионального использования нет (домашний) Производитель Raizer Производитель Рэйзр Гарантия 12 мес. Тип товара Беговая дорожка Двигатель (л.с.) 1 л.с. Размеры бегового полотна (см) 37х103 см Длина бегового полотна 103 см ручная (ступенчатая) Показания текущая скорость, расход энергии, пройденная дистанция Измерение пульса есть (встроенный датчик), кардиодатчик на ручке Удобства подставка для стакана, транспортировочные ролики/колеса, компенсаторы неровности пола Размер (ДхШхВ) 136x66x125 см"
45,Беговая дорожка Ferrum TM 1370D,,18420,Главная,Беговые дорожки,Ferrum,,,"Вес 36 кг Страна изготовления Китай Складной Да электрическая Максимальный вес пользователя 100 кг Ширина бегового полотна 37 см до 10 км/ч 3 уровня, устанавливается вручную Показания Скорость, время занятий, пройденное расстояние, количество израсходованных калорий, пульс Измерение пульса при помощи металлических датчиков на ручках Отображение Body Fat Оценка Body Fat; подставка для стакана, компенсаторы неровности пола Отображение текущей скорости да Отображение пройденного расстояния да Удобства Транспортировочные ролики + Встроенный лоток. Прочее Дополнительные опции Body Fat Производитель Ferrum Производитель Феррум Гарантия 6 мес. Тип товара Беговая дорожка Двигатель (л.с.) 1 л.с. Размеры бегового полотна (см) 37х103 Длина бегового полотна 103 см Максимальная скорость движения бегового полотна 10 км/ч Консоль 3-х экранная консоль Количество программ тренировки 9 Отображение пульса да Отображение расхода калорий да Отображение времени тренировки да есть Возможность профессионального использования нет (домашний) Размер (ДхШхВ) 136х66х125"


In [8]:
df

,title,article,price,cat_1,cat_2,cat_3,url,description,caracteristics
0,Беговая дорожка детская DFC VT-2301,,8990,Главная,Беговые дорожки,Беговая дорожка детская DFC VT-2301,https://petrasport.ru/d/begovaya-dorozhka-dets...,,Вес 9 кг Производитель ДиЭфСи Страна изготовл...
1,Беговая дорожка DFC SE-724,,10930,Главная,Беговые дорожки,Беговая дорожка DFC SE-724,https://petrasport.ru/d/DFC_SE-724.jpg,,Вес 27 кг Страна бренда Россия Гарантия 12 мес...
2,Беговая дорожка детская DFC VT-2301T,,10990,Главная,Беговые дорожки,Беговая дорожка детская DFC VT-2301T,https://petrasport.ru/d/begovaya-dorozhka-dets...,,Вес 11 кг Производитель ДиЭфСи Страна изготов...
3,Беговая дорожка DFC LV1005,,11630,Главная,Беговые дорожки,Беговая дорожка DFC LV1005,https://petrasport.ru/d/dfc_lv1005_01.jpg,,Вес 21 кг Страна бренда Россия Гарантия 12 мес...
4,Беговая дорожка HouseFit HT-9116HP,,11700,Главная,Беговые дорожки,Беговая дорожка HouseFit HT-9116HP,https://petrasport.ru/d/HouseFit_HT-9116HP.jpg,,Вес 26 кг Страна бренда Китай Гарантия 12 мес...
...,...,...,...,...,...,...,...,...,...
1731,Беговая дорожка Matrix T7XE VA (2013),,1349990,Главная,Беговые дорожки,Беговая дорожка Matrix T7XE VA (2013),https://petrasport.ru/d/Matrix_T7XE__VA_(2013)...,,"Вес 181 кг Страна бренда Johnson Health Tech, ..."
1732,Беговая дорожка TRUE C650,,1491400,Главная,Беговые дорожки,Беговая дорожка TRUE C650,https://petrasport.ru/d/begovaya_dorozhka_true...,,Вес 194 кг Страна изготовления Китай Тип това...
1733,Беговая дорожка TRUE Alpine Runner,,1669900,Главная,Беговые дорожки,Беговая дорожка TRUE Alpine Runner,https://petrasport.ru/d/begovaya_dorozhka_true...,,Вес 269 кг Страна изготовления Китай Тип това...
1734,Беговая дорожка MATRIX T7XI (T7XI-03),,1669990,Главная,Беговые дорожки,Беговая дорожка MATRIX T7XI (T7XI-03),https://petrasport.ru/d/begovaya_dorozhka_matr...,,"Вес 181 кг Страна бренда Johnson Health Tech, ..."
